In [1]:
import os
import glob
import pandas as pd
from datetime import datetime

In [2]:
directory_path = r"C:\Users\knut\Nextcloud\Geotechnik-Freigaben\Dienstleistung\Versuche_FuE\F_05_Schüttkegel_Anton\Versuche"

**Definitions**

In [3]:
def find_latest_csv_files(directory):
    """
    Findet die neueste CSV-Datei in jedem Unterordner des angegebenen Verzeichnisses.

    Parameter:
    directory (str): Der Pfad zum Hauptverzeichnis, in dem gesucht werden soll.

    Returns:
    dict: Ein Wörterbuch mit Ordnerpfaden als Schlüssel und den Pfaden der neuesten CSV-Dateien als Werte.
    """
    latest_files = {}

    # Durchlaufe alle Unterverzeichnisse
    for root, _, _ in os.walk(directory):
        # Suche nach allen .csv-Dateien im aktuellen Verzeichnis
        csv_files = glob.glob(os.path.join(root, '*.csv'))

        if csv_files:
            # Finde die neueste CSV-Datei im aktuellen Verzeichnis
            latest_csv = max(csv_files, key=os.path.getmtime)
            latest_files[root] = latest_csv

    return latest_files


*Load CSV Files* <br>
load all *.csv-files withing the directory_path and its sub directories <br>
and choose the latest *.csv within this subfolder <br>
it is assumed that this is the one with the most recent and correct data

In [4]:
latest_csv_files = find_latest_csv_files(directory_path)
latest_csv_files_list = list(latest_csv_files.items())
n = len(latest_csv_files_list)

df = pd.DataFrame(index=range(31))

# Pfad in seine Komponenten aufteilen
for i in range(n):
    path_parts = os.path.normpath(latest_csv_files_list[i][1]).split(os.sep)
    data = pd.read_csv(latest_csv_files_list[i][1],usecols=['Value'])
    new_row = pd.DataFrame({
    "Value": [path_parts[-4],path_parts[-3],path_parts[-2]]
    })
    data = pd.concat([new_row, data]).reset_index(drop=True)
    #df = data.copy()
    keyname =str(i)  
    df[keyname] = list(data.T.iloc[0])

row_names = pd.read_csv(latest_csv_files_list[0][1],usecols=['Parameter'])
new_row_names = pd.DataFrame({'Parameter' : ['KGV', 'Specimen', 'Test']})
row_names = pd.concat([new_row_names, row_names]).reset_index(drop=True)
df.index = row_names.T.iloc[0]

#data.loc[len(data)] = ["Subset", "SP-4-A", "-", "-"]
# # Verarbeitung der neuesten CSV-Dateien
# for folder, csv_file in latest_csv_files.items():
#     print(f"Neueste CSV in '{folder}': {csv_file}")
#     # Hier können Sie Ihren Code zur weiteren Verarbeitung der Dateien einfügen

In [5]:
# Bedingungen für die Tabs festlegen
df_transpose = df.T
categories = df_transpose['KGV'].unique()  # ['A', 'B']
#print(categories)
with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
    for category in categories:
        # DataFrame entsprechend der Bedingung filtern
        subset_df = df_transpose[df_transpose['KGV'] == category]
        subset_df.to_excel(writer, sheet_name=f'KGV_{category}', index=False)